In [1]:
# !pip install rank_bm25
from rank_bm25 import BM25Okapi
import json
import pdfplumber
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

#读取数据集
questions = json.load(open("questions.json",encoding="utf-8"))
pdf = pdfplumber.open("初赛训练数据集.pdf")
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })
    
#BM25
#对提问和PDF内容分词
question_words = [' '.join(jieba.lcut(x['question'])) for x in questions]
pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]
#提取bm25
bm25 = BM25Okapi(pdf_content_words)

for query_idx in range(len(questions)):
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idx = doc_scores.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

with open('submit_bm25_t1.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

for query_idx in range(len(questions)):
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idx = doc_scores.argsort()[::-1] + 1
    questions[query_idx]['reference'] = ['page_' + str(x) for x in max_score_page_idx[:10]]

with open('submit_bm25_t10.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\meetwin~\AppData\Local\Temp\jieba.cache
Loading model cost 0.652 seconds.
Prefix dict has been built successfully.
